In [9]:
using CSV
using DataFrames
using Statistics
# Ruta al archivo CSV
file_path = "Datos_Práctica_Evaluación_1.csv"

# Cargar los datos
data = CSV.read(file_path, DataFrame)

# Descripción de los datos
println("Descripción de los datos:")
println("Número de variables (columnas): ", size(data, 2)-2)
println("Número de instancias (filas): ", size(data, 1))
println("Número de individuos únicos: ", length(unique(data.subject)))
println("Número de clases de salida: ", length(unique(data.Activity)))
missing_percentage_per_column = map(col -> mean(ismissing, col) * 100, eachcol(data))
println("Porcentaje de nulos por variable:", missing_percentage_per_column)
println("Media de todos los porcentajes de nulo: ",sum(missing_percentage_per_column) / length(missing_percentage_per_column))


In [10]:
using CSV
using DataFrames
using Statistics
using MLJ  # Para técnicas de preprocesamiento

# Cargar datos
file_path = "C:\\Users\\leonf\\OneDrive\\Documentos\\AA_PRIMER_CUATRI\\MAAAI\\MAAA1\\Datos_Práctica_Evaluación_1.csv"
data = CSV.read(file_path, DataFrame)

# Rellenar valores faltantes
for col in names(data)
    if eltype(data[!, col]) <: Union{Missing, Number}  # Para columnas numéricas
        mean_value = mean(skipmissing(data[!, col]))  # Promedio de valores existentes
        if eltype(data[!, col]) <: Int  # Si la columna es de tipo entero
            mean_value = Int(round(mean_value))  # Redondear y convertir a entero
        end
        data[ismissing.(data[!, col]), col] .= mean_value  # Rellenar valores faltantes
    elseif eltype(data[!, col]) <: Union{Missing, AbstractString}  # Para columnas de texto
        mode_value = mode(skipmissing(data[!, col]))  # Valor más frecuente
        data[ismissing.(data[!, col]), col] .= mode_value
    end
end

println("Relleno de valores faltantes completado.")


# 3. Codificar variables categóricas
categorical_cols = filter(col -> eltype(data[!, col]) <: AbstractString, names(data))
println("Variables categóricas detectadas: ", categorical_cols)
for col in categorical_cols
    data[!, col] = categorical(data[!, col])  # Convertir a tipo categórico
end

# 4. Escalar variables numéricas
numerical_cols = filter(col -> eltype(data[!, col]) <: Number, names(data))
for col in numerical_cols
    col_values = data[!, col]
    data[!, col] = (col_values .- mean(col_values)) ./ std(col_values)  # Z-score
end

println("Transformación de datos completada.")

# 5. Verificar los datos transformados
println("Primeras filas de los datos:")
println(first(data, 5))
